In [1]:
import tensorflow as tf
import numpy as np

In [62]:
#计算Legendre多项式及其导数
def Legendre(x, m):
    L = np.zeros(np.max([m, 3]))
    L[0] = 1; L[1] = x; L[2] = 1 / 2 * (3 * x ** 2 - 1)
    if m > 3:
        for n in range(2, m - 1):
            L[n + 1] = (2 * n + 1) * x * L[n] / (n + 1)- n * L[n - 1] / (n + 1)
#    L = L[-1]
    return L
def Legendredx(x, m):
    L = np.zeros(np.max([m, 3]))
    L1 = Legendre(x, m)
    L[0] = 0; L[1] = 1; L[2] = 3 * x
    if m > 3:
        for n in range(2, m - 1):
            L[n + 1] = (2 * n + 1) * (L1[n] + x * L[n]) / (n + 1) - n * L[n - 1] / (n + 1)
#    L = L[-1]
    return L
def Legendred2x(x, m):
    L = np.zeros(np.max([m, 3]))
    L1 = Legendredx(x, m)
    L[0] = 0; L[1] = 0; L[2] = 3
    if m > 3:
        for n in range(2, m - 1):
            L[n + 1] = (2 * n + 1) * (2 * L1[n] + x * L[n]) / (n + 1) - n * L[n - 1] / (n + 1)
#    L = L[-1]
    return L

In [63]:
#计算勒让德多项式及其导数
def computeLegendrefunctionandderivation(X, m):
    L = len(X)
    L1 = []
    L2 = []
    L3 = []
    for i in range(L):
        temp = X[i]
        L1.append(Legendre(temp, m))
        L2.append(Legendredx(temp, m))
        L3.append(Legendred2x(temp, m))
    L1 = np.array(L1)
    L2 = np.array(L2)
    L3 = np.array(L3)
    value_dict = {'function': L1, 'firstderivation': L2, 'secondderivation': L3}
    return value_dict

In [73]:
#计算N(x,p)的导数
def firstdx_N(w, x, dx):
    #L = len(x)
    z = tf.reduce_sum(tf.multiply(tf.cast(w, tf.float64), tf.cast(x, tf.float64)))
    N = tf.nn.tanh(z)
    firstdx_N = (1 - tf.cast(N,tf.float64) ** 2) * tf.reduce_sum(tf.multiply(tf.cast(w,tf.float64), tf.cast(dx,tf.float64)))
    return firstdx_N
def seconddx_N(w, x, dx, dx2):
    z = tf.reduce_sum(tf.multiply(tf.cast(w, tf.float64), tf.cast(x, tf.float64)))
    N = tf.nn.tanh(z)
    seconddx_N = (1 - N ** 2) * (tf.reduce_sum(tf.multiply(tf.cast(w, tf.float64),tf.cast(dx2,tf.float64)))-\
                                2 * N *(tf.reduce_sum(tf.multiply(tf.cast(w,tf.float64), tf.cast(dx, tf.float64))))\
                                ** 2)
    #seconddx_N = (1 - N ** 2) * tf.reduce_sum(tf.multiply(tf.cast(w, tf.float64), tf.cast(dx2,tf.float64)))\
     #            - 2 * N * tf.reduce_sum(tf.multiply(tf.cast(w,tf.float64), tf.cast(dx, tf.float64)))
    return seconddx_N

In [74]:
#计算y=x**2*N(x,p)对x的导数
def dx(w, x, dx1):
    z = tf.reduce_sum(tf.multiply(tf.cast(w, tf.float64), tf.cast(x, tf.float64)))
    N = tf.nn.tanh(z)
    N1 = firstdx_N(w, x, dx1)
    dydx = 2 * x * N + N1 * (x ** 2)
    return dydx
def d2ydx2(w, x, dx, dx2):
    z = tf.reduce_sum(tf.multiply(tf.cast(w, tf.float64), tf.cast(x, tf.float64)))
    N = tf.nn.tanh(z)
    N1 = firstdx_N(w, x, dx)
    N2 = seconddx_N(w, x, dx, dx2)
    d2ydx2 = 2 * N + 4 * x * N1 + N2 * (x ** 2)
    return  d2ydx2

In [75]:
#计算代价函数
def costfunction(w, x, x0 , dx1, dx2):
    z = tf.reduce_sum(tf.multiply(tf.cast(w, tf.float64), tf.cast(x, tf.float64)))
    N = tf.nn.tanh(z)
    dy = dx(w, x, dx1)
    ddy2 = d2ydx2(w, x, dx1, dx2)
    costfunction = ddy2 + 2 / x0 * dy + 8 * tf.exp(x0 ** 2 * N) + 4 * tf.exp(x0 ** 2 * N / 2)
    return costfunction
def loss_function(X, w, Legendrefunction, firstdx, seconddx, length):
    L = []
    for i in range(length):
        temp0 = X[i]
        temp1 = Legendrefunction[i]
        temp2 = firstdx[i,:]
        temp3 = seconddx[i,:]
        temp = costfunction(w, temp1, temp0 , temp2, temp3)
        L.append(temp)
    #L = np.array(L)
    L = tf.reduce_sum(tf.multiply(L, L)) / 2
    return L

In [86]:
x = np.linspace(0.1, 1, 10)
n = 10
m = tf.constant(n,'int8')
value_dict = computeLegendrefunctionandderivation(x, n)
Legendrefunction = value_dict['function']
firstdx = tf.constant(value_dict['firstderivation'])
seconddx = tf.constant(value_dict['secondderivation'])

In [87]:
#建立输入层
x_train = tf.placeholder('float64', [None, n], name='x_train')

In [88]:
#建立输出层
W = tf.Variable(tf.truncated_normal([n, 1],stddev = 0.1), 'float64')
y_predict = tf.nn.tanh(tf.matmul(tf.cast(x_train, tf.float64), tf.cast(W, tf.float64)))

In [89]:
#定义损失函数
loss = loss_function(X = tf.constant(x,'float64'), w = W, 
                     Legendrefunction = x_train, firstdx = firstdx, seconddx = seconddx, length=n)

In [90]:
optimizer = tf.train.AdadeltaOptimizer(learning_rate=1).minimize(loss)

In [91]:
Epoch = 10000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(Epoch):
        sess.run(optimizer, feed_dict={x_train: Legendrefunction})
        #W = tf.Variable(tf.truncated_normal([n, 1], stddev=0.1), 'float')
        #loss = loss_function(X=tf.Variable(x, 'float'), m=m, w=W)
        #optimizer = tf.train.AdadeltaOptimizer(learning_rate=0.0001).minimize(loss)
        if (epoch % 100 == 0):
            print(sess.run(loss, feed_dict={x_train: Legendrefunction}))

16755.635924221977
4924.2760475293335
4674.108356958813
4673.874881899912
4673.874881899919
4674.463840082616
4675.846424870237
4677.708716687097
4679.490751355925
4681.236738225309
4682.943946319334
4684.61044241066
4686.236991686169
4687.825383875236
4689.377390131101
4690.8950716542095
4692.3805973855515
4693.835810646613
4695.261711770006
4696.660892246724
4698.034384301228
4699.383530776295
4700.709560996736
4702.013743997521
4703.2973989168895
4704.560994067537
4705.805658896332
4707.031935462901
4708.241624117642
4709.434450957426
4710.611712449996
4711.773620219179
4712.920360459657
4714.053264556198
4715.17295114964
4716.279183635251
4717.373610863577
4718.4547329729385
4719.524720122028
4720.583239147998
4721.630906478614
4722.667879538511
4723.694544377251
4724.710990827374
4725.71767992049
4726.71492650431
4727.702828402877
4728.681721282664
4729.652106499676
4730.61467527274
4731.568459723862
4732.514450616918
4733.452759153357
4734.3830249181865
4735.305723266414
4736.221

In [38]:
Legendrefunction.dtype

dtype('float64')

In [56]:
W